In [ ]:
# Standard libraries
import json
import math
import os
import random
from typing import (Any, Dict, List, Tuple)

# Third party libraries
import IPython
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import scipy.io.wavfile
import scipy.signal
import sklearn
import sklearn.tree

print('IPython.__version__ = %s' % IPython.__version__)
print('matplotlib.__version__ = %s' % matplotlib.__version__)
print('numpy.__version__ = %s' % np.__version__)
print('pandas.__version__ = %s' % pd.__version__)
print('scipy.__version__ = %s' % scipy.__version__)
print('sklearn.__version__ = %s' % sklearn.__version__)

print('\nseeding random with %d\n' % int("9fe3ddf4da76a6", 16))
random.seed(int("9fe3ddf4da76a6", 16))

with open('/proc/%d/status' % os.getpid(), 'rb') as f:
    print(f.read().decode('utf-8'))

In [ ]:
# Raw audio: ../audios/*.wav
# Labels: ../outputs/*.json --> ../labels/*.tsv
data_files = []
training_files = set([])
eval_files = set([])

# sampling? first-N-seconds? last-N-seconds? specific time-window?
# lazy loading? mono-channel?

# Viewing options:
# 1) Signal amplitude
# 2) Test signal amplitude (examples: sum(freqs[:5]), sum(freqs[5:10]), ...)
# 3) Spectrogram pcolormesh
# 4) IPython.display.Audio
# 5) JSON utterance labels
# 6) Time series labels, i.e. for (1-3), `plt.axvline(x=item['start'], color='#d62728')`, TODO: linewidth=wut?
# `%matplotlib notebook` may be handy?

# want: frames (aka windows) of 10 ms, steps of 5 ms.
# datastruct: (file_name, frame index, signal_rate (example: 44.1kHz), raw_signal_vec, freqs_vec (further want: speech_vec + background_vec), label, phoneme)
df = pd.DataFrame(data={
    'file_name': ['a', 'a', 'a', 'b', 'b'],
    'frame_index': [0, 1, 2, 0, 1],
    'signal_rate': [44100, 44100, 44100, 44100, 44100],
    'window_size': [441, 441, 441, 441, 441],
    'step_size': [220, 220, 220, 220, 220],
    'window_max_i': [0, 0, 1, 0, 1],
    # These are a bit misleading because their length is 2, but window_size says
    # they should be 441.
    'raw_signal_vec': [[0, 0], [1, 1], [1, 2], [0, 0], [0, 1]],
    'freqs_vec': [[0, 0], [1, 0], [2, 1], [0, 0], [0, 1]],
    # TODO: fft(fft(raw_signal)) b/c harmonics. consider librosa's "pitch class"
    'label': [0, 0, 1, None, None],
    'phoneme': [None, None, 'a', None, None],
})
df